In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hindi-english-parallel-corpus/hindi_english_parallel.csv


In [2]:
pip install -q bpemb

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install -q torchsummary

Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch 
import torch.nn as nn
from bpemb import BPEmb
from torchsummary import summary
from tqdm import tqdm
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
data = pd.read_csv('/kaggle/input/hindi-english-parallel-corpus/hindi_english_parallel.csv')
data.head()

,hindi,english
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,Give your application an accessibility workout
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,Accerciser Accessibility Explorer
2,निचले पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the bottom panel
3,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the top panel
4,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...,A list of plugins that are disabled by default


In [6]:
data.dropna(inplace=True)
data.drop_duplicates(inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1353877 entries, 0 to 1561839
Data columns (total 2 columns):
 #   Column   Non-Null Count    Dtype 
---  ------   --------------    ----- 
 0   hindi    1353877 non-null  object
 1   english  1353877 non-null  object
dtypes: object(2)
memory usage: 31.0+ MB


In [7]:
data['english_len'] = data['english'].apply(lambda x:len(x.split()))
data['hindi_len'] = data['hindi'].apply(lambda x:len(x.split()))
data.head()

,hindi,english,english_len,hindi_len
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,Give your application an accessibility workout,6,8
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,Accerciser Accessibility Explorer,3,3
2,निचले पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the bottom panel,8,7
3,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the top panel,8,7
4,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...,A list of plugins that are disabled by default,9,12


In [8]:
data = data[(data.english_len>=5) & (data.english_len<=15) & (data.hindi_len>=5) & (data.hindi_len<=15)]
data.head()

,hindi,english,english_len,hindi_len
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,Give your application an accessibility workout,6,8
2,निचले पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the bottom panel,8,7
3,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the top panel,8,7
4,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...,A list of plugins that are disabled by default,9,12
6,पहुंचनीय आसंधि (नोड) को चुनते समय हाइलाइट बक्स...,The duration of the highlight box when selecti...,10,10


In [9]:
data = data.sample(n=25000, random_state=0)
train_split, test_split = train_test_split(data, test_size=0.1, random_state=0)
train_split = train_split.reset_index(0).drop(['index'], axis=1)
test_split = test_split.reset_index(0).drop(['index'], axis=1)

In [10]:
print(len(train_split), len(test_split))
train_split.head()

22500 2500


,hindi,english,english_len,hindi_len
0,एक खूब छोटा अंश जिस में तत्वों के गुण होते है।,A very small component acquiring a quality of ...,9,11
1,कोटा जानकारी समर्थित नहीं फ़ोल्डर '% s' के लिए,No IMAP mailbox available for folder '% s',8,9
2,परन्तु यह शीघ्र ही अपर्याप्त प्रतीत हुआ...।,But soon this was found rather inadequate.,7,7
3,उसने गिरफ्तार व्यक्ति के लिए प्रतिभू की भूमिका...,He acted as ad - promisor for the arrested per...,10,9
4,अल्लाह से क्षमा की प्रार्थना करो। निस्संदेह अल...,Ask God for forgiveness: He is most forgiving ...,10,12


## Tokenization approaches
Tokenization is of two types:
1)character level
2)word level

In [11]:
# this is basically the module by which we're gonna perform tokenization according to the Byte-Level Byte Pair Encoding
bpemb_en = BPEmb(lang='en')
bpemb_hi = BPEmb(lang='hi')

downloading https://nlp.h-its.org/bpemb/en/en.wiki.bpe.vs10000.model


100%|██████████| 400869/400869 [00:01<00:00, 389695.29B/s]


downloading https://nlp.h-its.org/bpemb/en/en.wiki.bpe.vs10000.d100.w2v.bin.tar.gz


100%|██████████| 3784656/3784656 [00:01<00:00, 2160415.10B/s]


downloading https://nlp.h-its.org/bpemb/hi/hi.wiki.bpe.vs10000.model


100%|██████████| 463471/463471 [00:01<00:00, 459389.69B/s]


downloading https://nlp.h-its.org/bpemb/hi/hi.wiki.bpe.vs10000.d100.w2v.bin.tar.gz


100%|██████████| 3796553/3796553 [00:01<00:00, 1975583.13B/s]


## dataset and dataloader

In [12]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data, max_seq_len=64):
        super(CustomDataset, self).__init__()
        self.data = data
        self.max_seq_len = max_seq_len
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        eng_sen = self.data.english.iloc[index]
        hin_sen = self.data.hindi.iloc[index]
        eng_tokens = bpemb_en.encode_ids_with_bos_eos(eng_sen)
        hin_tokens = bpemb_hi.encode_ids_with_bos_eos(hin_sen)
        trg_input_tokens = hin_tokens[:-1]
        trg_output_tokens = hin_tokens[1:]
        
        eng_mask = [1]*(len(eng_tokens))
        hin_mask = [1]*(len(trg_input_tokens))
        
        eng_tokens = eng_tokens + [0]*(self.max_seq_len - len(eng_tokens))
        trg_input_tokens = trg_input_tokens + [0]*(self.max_seq_len - len(trg_input_tokens))
        trg_output_tokens = trg_output_tokens + [0]*(self.max_seq_len - len(trg_output_tokens))
        
        eng_mask = eng_mask + [0]*(self.max_seq_len - len(eng_mask))
        hin_mask = hin_mask + [0]*(self.max_seq_len - len(hin_mask))
        # pad eng_tokens upto max_seq_len
        # pad_hin_tokens upto max_seq_len
        # then create masks for both of the inputs, and make them upto the dimension needed
        
        # now we have to pad the sequence upto max length and create the masks for the same
        
        
        return torch.tensor(eng_tokens), torch.tensor(trg_input_tokens), torch.tensor(trg_output_tokens), torch.tensor(eng_mask), torch.tensor(hin_mask)

## encoder , decoder and attention mechanism

In [13]:
def scaled_dot_product_attention(query, key, value, mask=None):
    # this is going in the single head self-attention.
    # query, key, value shape will be (b, h, t, d/h) 
    d_k = query.shape[-1]
    scaled_scores = torch.matmul(query, torch.transpose(key, -2, -1))/np.sqrt(d_k)  # shape is  (b, h, t, t)
    
    if mask is not None:
        # mask must be of shape (b,h,t,t)
        scaled_scores = torch.where(mask==0, -np.inf, scaled_scores)
        
    weights = torch.nn.Softmax(dim=-1)(scaled_scores) # shape is (b,h,t,t)
    return torch.matmul(weights, value)  # shape will be (b,h,t,d/h)

In [14]:
# this is an example of how multi-head works
b = 1
t = 3
d = 12
h = 3
# first assigning the weights
wq0 = torch.rand((d, d//h))
wk0 = torch.rand((d, d//h))
wv0 = torch.rand((d, d//h))

wq1 = torch.rand((d, d//h))
wk1 = torch.rand((d, d//h))
wv1 = torch.rand((d, d//h))

wq2 = torch.rand((d, d//h))
wk2 = torch.rand((d, d//h))
wv2 = torch.rand((d, d//h))

# let us take some random input also 
x = torch.rand((1, t, d))

# now calculating q, k, v for each attention head
q0 = torch.matmul(x, wq0)
k0 = torch.matmul(x, wk0)
v0 = torch.matmul(x, wv0)

q1 = torch.matmul(x, wq1)
k1 = torch.matmul(x, wk1)
v1 = torch.matmul(x, wv1)

q2 = torch.matmul(x, wq2)
k2 = torch.matmul(x, wk2)
v2 = torch.matmul(x, wv2)

# now we have to perform scaled self-attention on each head
op0 = scaled_dot_product_attention(q0, k0, v0)
op1 = scaled_dot_product_attention(q1, k1, v1)
op2 = scaled_dot_product_attention(q2, k2, v2)


# now the final ouputu is the conactenation of all the three outputs
op = torch.concat([op0, op1, op2], dim=2)
print(op.shape)
print(op)

torch.Size([1, 3, 12])
tensor([[[4.2392, 3.9870, 4.9238, 3.2736, 3.2142, 3.9222, 4.1703, 4.6155,
          4.0666, 4.7017, 3.9342, 3.5006],
         [4.2389, 3.9866, 4.9229, 3.2733, 3.2142, 3.9221, 4.1702, 4.6154,
          4.0466, 4.6776, 3.9209, 3.4825],
         [4.2383, 3.9859, 4.9217, 3.2727, 3.2140, 3.9218, 4.1700, 4.6153,
          4.0105, 4.6341, 3.8968, 3.4500]]])


In [15]:
# but we don't have to do this multi-headed attention by calculating different heads, so let us first of all 
# concatenate the weights of the self attention 
wq = torch.concat([wq0, wq1, wq2], axis=1)
wk = torch.concat([wk0, wk1, wk2], axis=1)
wv = torch.concat([wv0, wv1, wv2], axis=1)

# now we have to get the q,k ,v
q = torch.matmul(x, wq)
k = torch.matmul(x, wk)
v = torch.matmul(x, wv)

# and finally we have to do the self-attention 
print(q0, '\n')
print(q1, '\n')
print(q2, '\n')
print(q)

tensor([[[3.9438, 3.8335, 4.6411, 4.9659],
         [3.3887, 2.1897, 3.6973, 3.6439],
         [2.6640, 2.4181, 3.1155, 3.0308]]]) 

tensor([[[4.4655, 6.2410, 4.1343, 4.0531],
         [3.6420, 3.6887, 2.8314, 3.2289],
         [3.0581, 3.5235, 2.6255, 2.7128]]]) 

tensor([[[3.8300, 3.3751, 3.0031, 5.1045],
         [3.0232, 2.5517, 2.0502, 3.5196],
         [2.8088, 1.7579, 1.7806, 3.1562]]]) 

tensor([[[3.9438, 3.8335, 4.6411, 4.9659, 4.4655, 6.2410, 4.1343, 4.0531,
          3.8300, 3.3751, 3.0031, 5.1045],
         [3.3887, 2.1897, 3.6973, 3.6439, 3.6420, 3.6887, 2.8314, 3.2289,
          3.0232, 2.5517, 2.0502, 3.5196],
         [2.6640, 2.4181, 3.1155, 3.0308, 3.0581, 3.5235, 2.6255, 2.7128,
          2.8088, 1.7579, 1.7806, 3.1562]]])


In [16]:
# now we have to do operations on q such that we get q0, q1, q2 from q
q = q.reshape(b, t, h, d//h).permute(0, 2, 1, 3)
k = k.reshape(b, t, h, d//h).permute(0, 2, 1, 3)
v = v.reshape(b, t, h, d//h).permute(0, 2, 1, 3)

op_multi = scaled_dot_product_attention(q, k, v)
op, op_multi

(tensor([[[4.2392, 3.9870, 4.9238, 3.2736, 3.2142, 3.9222, 4.1703, 4.6155,
           4.0666, 4.7017, 3.9342, 3.5006],
          [4.2389, 3.9866, 4.9229, 3.2733, 3.2142, 3.9221, 4.1702, 4.6154,
           4.0466, 4.6776, 3.9209, 3.4825],
          [4.2383, 3.9859, 4.9217, 3.2727, 3.2140, 3.9218, 4.1700, 4.6153,
           4.0105, 4.6341, 3.8968, 3.4500]]]),
 tensor([[[[4.2392, 3.9870, 4.9238, 3.2736],
           [4.2389, 3.9866, 4.9229, 3.2733],
           [4.2383, 3.9859, 4.9217, 3.2727]],
 
          [[3.2142, 3.9222, 4.1703, 4.6155],
           [3.2142, 3.9221, 4.1702, 4.6154],
           [3.2140, 3.9218, 4.1700, 4.6153]],
 
          [[4.0666, 4.7017, 3.9342, 3.5006],
           [4.0466, 4.6776, 3.9209, 3.4825],
           [4.0105, 4.6341, 3.8968, 3.4500]]]]))

In [17]:
op_multi.permute(0, 2, 1, 3).reshape(b, t, d)==op

tensor([[[True, True, True, True, True, True, True, True, True, True, True,
          True],
         [True, True, True, True, True, True, True, True, True, True, True,
          True],
         [True, True, True, True, True, True, True, True, True, True, True,
          True]]])

## model training 

In [18]:
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadSelfAttention, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        
        # let us say that our input is of shape (b, t, d) 
        self.wq = nn.Linear(in_features = self.d_model, out_features = self.d_model, bias = False) 
        self.wk = nn.Linear(in_features = self.d_model, out_features = self.d_model, bias = False)
        self.wv = nn.Linear(in_features = self.d_model, out_features = self.d_model, bias = False)
        # remember that wq, wk, ev defined above are just the matrices nothing more
        
        # now the final dense layer to add some nolinearity in it 
        self.fc = nn.Linear(self.d_model, self.d_model)
        
    def forward(self, q, k, v, mask=None):
        # shape of q, k, v is (b, t, d)
        q = self.wq(q).reshape(q.shape[0], q.shape[1], self.num_heads, self.d_model//self.num_heads).permute(0, 2, 1, 3) 
        k = self.wk(k).reshape(k.shape[0], k.shape[1], self.num_heads, self.d_model//self.num_heads).permute(0, 2, 1, 3) 
        v = self.wv(v).reshape(v.shape[0], v.shape[1], self.num_heads, self.d_model//self.num_heads).permute(0, 2, 1, 3) 
        
        # now shape of q, k, v is (b, h, t, d/h)
        # now we have to simply perform  self-attention for every head
        op = scaled_dot_product_attention(q, k, v, mask)  # shape of op is (b, h, t, d/h)
        op = op.permute(0, 2, 1, 3)
        op = op.reshape(op.shape[0], op.shape[1], self.d_model)  # now shape of op is (b, t, d)
        return self.fc(op) # shapes are (b, t, d)

In [19]:
class feedforward(nn.Module):
    def __init__(self, d_model, hidden_dim):
        super(feedforward, self).__init__()
        self.fc1 = nn.Linear(d_model, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, d_model)
        
    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

In [20]:
class encoder_block(nn.Module):
    def __init__(self, d_model, num_heads, hidden_dim, dropout_rate=0.1):
        super(encoder_block, self).__init__()
        self.mhsa = MultiHeadSelfAttention(d_model, num_heads)
        self.fc = feedforward(d_model, hidden_dim) # gonna give the shapes again to be (b, t, d)
        
        self.dropout1 = nn.Dropout(p=dropout_rate)
        self.dropout2 = nn.Dropout(p=dropout_rate)
        
        self.layernorm1 = nn.LayerNorm(d_model)
        self.layernorm2 = nn.LayerNorm(d_model)
    
    def forward(self, x, mask=None):
        # input shape is (b, t, d)
        op = self.mhsa(x, x, x, mask) # op shape is (b, t, d) and attention weights are of shape (b, t, t)
        op = self.dropout1(op)
        # now we have to pass it through layer normalization (study it)
        op = self.layernorm1(op + x)
        ffn_op = self.fc(op)
        ffn_op = self.dropout2(ffn_op)
        op = self.layernorm2(op + ffn_op)
        return op

In [21]:
class encoder_transformer(nn.Module):
    def __init__(self, num_blocks, d_model, num_heads, hidden_dim, src_vocab_size, max_seq_len, dropout_rate=0.1):
        # max_seq_len is the number of time steps, i'll be referring it as t
        super(encoder_transformer, self).__init__()
        self.num_blocks = num_blocks
        self.d_model = d_model
        self.num_heads = num_heads
        self.hidden_dim = hidden_dim 
        self.vocab_size = src_vocab_size
        self.max_seq_len = max_seq_len
        
        self.token_embeds = nn.Embedding(src_vocab_size, d_model)
        self.pos_embeds = nn.Embedding(max_seq_len, d_model)
        self.dropout = nn.Dropout(p = dropout_rate)
        self.blocks = nn.ModuleList([encoder_block(d_model, num_heads, hidden_dim, dropout_rate=dropout_rate) 
                      for _ in range(self.num_blocks)])
        
            
        
        
    
    def forward(self, source, mask=None):
        # shape of source is (b, t)
        # all source sentences will be padded and padding will be static
        source = source.type(torch.LongTensor).to(device)
        # comment out above line for 
        t_embeds = self.token_embeds(source) # (b, t, d)
        pos_ids = torch.broadcast_to(torch.arange(self.max_seq_len), (x.shape[0], self.max_seq_len)).type(torch.LongTensor
                                                                                                         ).to(device)
        p_embeds = self.pos_embeds(pos_ids) # (b, t, d)
        
        inp = t_embeds + p_embeds  # (b, t, d)
        op = self.dropout(inp)  # (b, t, d)
        
        for _, block in enumerate(self.blocks):
            op = block(op, mask)
            
        return op

## decoder

In [22]:
class decoder_block(nn.Module):
    def __init__(self, d_model, num_heads, hidden_dim, dropout_rate=0.1):
        super(decoder_block, self).__init__()
        self.mhsa1 = MultiHeadSelfAttention(d_model, num_heads)
        self.mhsa2 = MultiHeadSelfAttention(d_model, num_heads)
        
        self.fc = feedforward(d_model, hidden_dim)
        
        self.dropout1 = nn.Dropout(p=dropout_rate)
        self.dropout2 = nn.Dropout(p=dropout_rate)
        self.dropout3 = nn.Dropout(p=dropout_rate)
        
        self.layernorm1 = nn.LayerNorm(d_model)
        self.layernorm2 = nn.LayerNorm(d_model)
        self.layernorm3 = nn.LayerNorm(d_model)
        
        
    def forward(self, encoder_ouptut, target, decoder_mask=None, memory_mask=None):
        mhsa_op1 = self.mhsa1(target, target, target, decoder_mask)
        mhsa_op1 = self.dropout1(mhsa_op1)
        mhsa_op1 = self.layernorm1(mhsa_op1 + target)
        
        mhsa_op2 = self.mhsa2(mhsa_op1, encoder_ouptut, encoder_ouptut, memory_mask)
        mhsa_op2 = self.dropout2(mhsa_op2)
        mhsa_op2 = self.layernorm2(mhsa_op1 + mhsa_op2)
        
        fc_op = self.fc(mhsa_op2)
        fc_op = self.dropout3(fc_op)
        op = self.layernorm3(fc_op + mhsa_op2)
        
        return op

In [23]:
class decoder_transformer(nn.Module):
    def __init__(self, num_blocks, d_model, num_heads, hidden_dim, trg_vocab_size, max_seq_len, dropout_rate=0.1):
        super(decoder_transformer, self).__init__()
        self.token_embeds = nn.Embedding(trg_vocab_size, d_model)
        self.pos_embeds = nn.Embedding(max_seq_len, d_model)
        self.max_seq_len = max_seq_len
        self.dropout = nn.Dropout(p=dropout_rate)
        
        self.blocks = nn.ModuleList([decoder_block(d_model, num_heads, hidden_dim, dropout_rate) 
                                    for _ in range(num_blocks)])
    
    def forward(self, encoder_output, target, decoder_mask=None, memory_mask=None):
        # shape of target is (b, t)
        target = target.type(torch.LongTensor).to(device)
        t_embeds = self.token_embeds(target) # (b, t, d)
        pos_ids = torch.broadcast_to(torch.arange(self.max_seq_len), (x.shape[0], self.max_seq_len)).type(torch.LongTensor
                                                                                                         ).to(device)
        p_embeds = self.pos_embeds(pos_ids)  # (b, t, d)
        
        inp = t_embeds + p_embeds
        op = self.dropout(inp)
        # now op is (b, t, d)
        
        for _, block in enumerate(self.blocks):
            op = block(encoder_output, op, decoder_mask, memory_mask)
            
        return op

In [24]:
class transformer(nn.Module):
    def __init__(self, num_blocks, d_model, num_heads, hidden_dim, src_vocab_size, trg_vocab_size, max_seq_len, 
                dropout_rate=0.1):
        super(transformer, self).__init__()
        self.encoder = encoder_transformer(num_blocks, d_model, num_heads, hidden_dim, src_vocab_size, max_seq_len,
                            dropout_rate=0.1)
        self.decoder = decoder_transformer(num_blocks, d_model, num_heads, hidden_dim, src_vocab_size, max_seq_len,
                            dropout_rate=0.1)
        
        self.fc = nn.Linear(d_model, trg_vocab_size)
        self.lookahead = torch.tril(torch.ones((max_seq_len, max_seq_len))).to(device)
    
    def forward(self, src, trg, src_pad_mask=None, trg_pad_mask=None):
        # we require masks to be of shape (b, 1, 1, t)
        dec_mask = None
        if src_pad_mask is not None:
            src_pad_mask = src_pad_mask.unsqueeze(1).unsqueeze(1)
            src_pad_mask = src_pad_mask.to(device)
        if trg_pad_mask is not None:
            trg_pad_mask = trg_pad_mask.unsqueeze(1).unsqueeze(1)
            dec_mask = torch.minimum(trg_pad_mask, self.lookahead)
            dec_mask = dec_mask.to(device)
            
        enc_op = self.encoder(src, src_pad_mask)
        op = self.decoder(enc_op, trg, dec_mask, src_pad_mask) # op is of shape (b, t, d)
        # and generate the required look ahead mask
        op = op.reshape(-1, op.shape[-1]) # shape is (b*t, d)
        op = self.fc(op) # shape is (b*t, trg_vocab_size)
        
        return op

## utils 

In [25]:
def BLEU():
    pass

In [26]:
def METEOR():
    pass

In [27]:
def save_checkpoint(state, filename='my_checkpoint.pth'):
    # will save model and optimizer params at every epoch
    print("-> Saving CheckPoint")
    torch.save(state, filename)

In [28]:
def load_checkpoint(checkpoint, model):
    # it will just load, we can train it further, make changes to the architecture
    # and simply use it to predict
    print("-> Loading CheckPoint")
    model.load_state_dict(checkpoint["state_dict"])

In [29]:
def train(loader, model, optimizer, scaler, scheduler, loss_fn, epoch, device=device):
    '''
    it is the training procedure for one epoch of the network
    '''
    losses = 0
    model.train()
    num_batches = len(loader)
    batches = tqdm(loader) # tqdm will be used to generate progress bars
    for idx, batch in enumerate(batches, 0):
        src = batch[0].to(device)  # (batch_size, max_len)
        trg_inp = batch[1].to(device)  # (batch_size, max_len)
        trg_op = batch[2].to(device) # (batch_size, max_len)
        src_pad_mask = batch[3].to(device) # (batch_size, max_len)
        trg_pad_mask = batch[4].to(device) # (batch_size, max_len)

        # forward
        optimizer.zero_grad()
        with torch.cuda.amp.autocast(): # for gradient underflowing and overflowing and it makes training faster by converting all floats to float16
            op = model(src, trg_inp, src_pad_mask, trg_pad_mask) # op shape is (batch_size*max_len, trg_vocab_size+1)
            trg_op = trg_op.reshape(trg_op.shape[0]*trg_op.shape[1]) # trg_op shape is (batch_size*max_len)
            loss = loss_fn(op, trg_op) # loss_fn should contain the parametere ignore_idx=0, so that 
            # losses corresponding to the padding token isn't calculated

        # making all the previous gradients zero 
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        batches.set_postfix(loss = loss.item(), epoch=epoch) # loss of this current batch on current iteration 
        losses+= loss.item()

    losses/=num_batches    
    #scheduler.step()
    return losses

In [30]:
def test():
    # i will only use this function for measuring its accuracy on different metrics
    # for this task such as meteor and bleu
    pass

In [31]:
def translate_sentence(sentence, model, max_seq_len, trans_len=50):
    model.eval()
    # first i am given the english sentence
    inp = bpemb_en.encode_ids_with_bos_eos(sentence)
    enc_mask = [1]*(len(inp))
    inp = inp + [0]*(max_seq_len - len(inp))
    enc_mask = enc_mask + [0]*(max_seq_len - len(enc_mask))
    # inp shape is (max_seq_len) and so is of mask
    #print(f'input = {inp}\n\nmask = {enc_mask}')
    inp = torch.tensor(inp).unsqueeze(0).to(device) # shape of input is (1, max_seq_len)
    enc_mask = torch.tensor(enc_mask).unsqueeze(0).to(device) # shape of mask is also (1, max_seq_len)
    #inp =  inp.unsqueeze(0)
    #enc_mask = enc_mask.unsqueeze(0).unsqueeze(1).unsqueeze(1) #  shape should be (b,1,1,max_seq_len)
    # now the shapes are as required by the transformer
    
    #enc_op = model.encoder(inp, enc_mask)
    # now we have to decode the sentence one-by-one 
    # so let us first of all make the inputs and the corresponding trg_mask
    trg_inp = torch.zeros(max_seq_len).unsqueeze(0).to(device) # shape is (1, max_seq_len)
    trg_mask = torch.zeros(max_seq_len).unsqueeze(0).to(device) # shape is (1, max_seq_len)
    
    #lookahead = torch.tril(torch.ones((max_seq_len, max_seq_len))).to(device) 
    
    trg_inp[0, 0] = 1 # 1 means <sos> token
    translation = []
    #trg_mask[0, 0, 0, len(translation)] = 1 # as trg_inp has only one word in it at current time step
    #dec_mask = torch.minimum(trg_mask, lookahead)
    trg_mask[0, len(translation)] = 1 # as trg_inp has only one word in it at current time step
    last_token = trg_inp[0, len(translation)]
    # now we have to pass it through a decoder until we get a <eos> token or we exceed trans_len
    while len(translation)<trans_len and last_token!=2: # 2 means <eos> token
        # now we have to pass the above inputs through decoder
        #dec_op = model.decoder(enc_op, trg_inp, dec_mask, enc_mask) 
        # now shape of decoder op will be 
        # shape of output is (1, hindi_vocab_size)
        op = model(inp, trg_inp, enc_mask, trg_mask) # shape of op will be (max_seq_len, trg_vocab_size)
        op = op.argmax(dim=1)  # shape will be (max_seq_len)
        last_token = op[len(translation)].item()
        translation.append(last_token)
        trg_inp[0, len(translation)] = last_token  # updating the last token in the trg_inp
        trg_mask[0, len(translation)] = 1  # setting up the mask for the current value equal to 1
        #print(f"DONE {len(translation)} times")
        
    model.train()
    return bpemb_hi.decode(translation)

## driver code

In [32]:
# training hyperparameters
num_epochs =  20
lr = 3e-4
batch_size = 64

train_dataset = CustomDataset(train_split, max_seq_len=64)
test_dataset = CustomDataset(test_split, max_seq_len=64)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [33]:
# model hyperparameters
num_blocks = 3
d_model = 512
num_heads = 8
hidden_dim = 4*d_model
src_vocab_size = bpemb_en.vocab_size + 1 # +1 due to padding token
trg_vocab_size = bpemb_hi.vocab_size + 1 # +1 due to padding token
max_seq_len = 64

# testing the model
model = transformer(num_blocks, d_model, num_heads, hidden_dim, src_vocab_size, 
                              trg_vocab_size, max_seq_len).to(device)
summary(model, [(max_seq_len, ), (max_seq_len ,)])

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         Embedding-1              [-1, 64, 512]       5,120,512
         Embedding-2              [-1, 64, 512]          32,768
           Dropout-3              [-1, 64, 512]               0
            Linear-4              [-1, 64, 512]         262,144
            Linear-5              [-1, 64, 512]         262,144
            Linear-6              [-1, 64, 512]         262,144
            Linear-7              [-1, 64, 512]         262,656
MultiHeadSelfAttention-8              [-1, 64, 512]               0
           Dropout-9              [-1, 64, 512]               0
        LayerNorm-10              [-1, 64, 512]           1,024
           Linear-11             [-1, 64, 2048]       1,050,624
             ReLU-12             [-1, 64, 2048]               0
           Linear-13              [-1, 64, 512]       1,049,088
      feedforward-14              [

In [34]:
# let us test our model on some actual input to check it doesn't break
for _, batch in enumerate(train_loader):
    src = batch[0][0].unsqueeze(0).to(device)
    trg_inp = batch[1][0].unsqueeze(0).to(device)
    trg_op = batch[2][0].unsqueeze(0).to(device)
    src_pad_mask = batch[3][0].unsqueeze(0).to(device)
    trg_pad_mask = batch[4][0].unsqueeze(0).to(device)
    break
    
#print(f'{src}\n\n{src_pad_mask}\n\n{trg}\n\n{trg_pad_mask}')
op = model(src, trg_inp, src_pad_mask, trg_pad_mask)
op = nn.Softmax(dim=-1)(op)
out = torch.max(op, dim=-1).indices
print(f'Original English Sentence {bpemb_en.decode(src.tolist())}\n\nOriginal Hindi Sentence {bpemb_hi.decode(trg_inp.tolist())}\n\nPredicted Sentence {bpemb_hi.decode(out.tolist())}')
# ?? are because of the padding tokens we can easily remove them when needed

Original English Sentence ['and whosoever is in the earth, altogether, so that it might save him. ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇ ']

Original Hindi Sentence ['और जितने आदमी ज़मीन पर हैं सब को ले ले और उसको छुटकारा दे दें ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇ ']

Predicted Sentence ['बिन', 'पथ', 'माइक्र', 'हिस्से', '’', 'गुलाब', 'ब्दिक', 'class', 'अर', 'छुट', 'स्कॉ', 'वाइ', 'खंड', 'दिख', 'ज्ञात', 'पारस्परिक', 'एसी', 'णे', 'उसी', 'चंडी', 'सरकारों', 'तीत', '0000–00', 'हिस', 'rec', 'थियेटर', '१६', 'लाह', '१६', 'डे', 'भि', 'दिन', '१६', 'धूम्रपान', 'उम्मीदवार', 'छू', 'बॉक्स', '१६', 'पहली', 'मोटे', 'bl', 'पेंट', 'बेटा', '१६', 'दिन', 'ख्त', 'इण्डिया', 'chur', '१६', 'आध्यात्म', 'chur', 'किन', 'चार', 'ोत्सव', 'char', 'सूचक', 'बिहारी', 'पड़ोसी', 'नैनी', 'बैंड', 'टीय', 'रथ', 'तालाब', 'अनुमति']


In [35]:
# exampel of translated sentence
text = 'another plant, the aluminium corporation of india, came into existence after the war.'
print(translate_sentence(text, model, 64))

['डाला', 'an', '६६७६', 'विजे', 'गढ़वाल', 'हिट', 'टॉप', 'िर', '२००१', 'ताकि', 'आहार', 'िपि', 'वालों', 'पाठक', 'अमृत', 'पारस्परिक', 'माइक्रोस', 'शिश', 'प्रारम्भिक', 'उतार', '्यालय', 'संगठ', 'राज', 'कृषि', '&', 'निर्धारण', 'ऑनलाइन', 'उपभोक्ता', 'उतार', 'उगा', 'चिकित्सक', 'प्रवासी', 'न्यूटन', 'कान', 'मेजर', '))', 'les', 'साइट', 'विश्वविद्यालयों', 'शुर', 'कर्ण', 'गवर्नर', 'हेतु', 'हाँ', 'वाद्य', 'दोनो', 'प्रतिपादन', 'emb', 'जहाँ', 'मार्']


In [36]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_fn = nn.CrossEntropyLoss(ignore_index=0) 
scaler = torch.cuda.amp.GradScaler() 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
losses = []
# for inference
sentences = ['another plant, the aluminium corporation of india, came into existence after the war.', 
             'He is doing very good these days', 'this guy is totally mad', 'what were you saying that day?']

/tmp/ipykernel_155/788890195.py:3: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


In [37]:
# taking the test sentences for checking how good the model is trained
def infer(sentences, model, max_seq_len):
    for idx, sentence in enumerate(sentences):
        print(f'Example {idx+1}:\n{sentence}\n{translate_sentence(sentence, model, max_seq_len)}\n\n')

In [38]:
infer(sentences, model, max_seq_len)

Example 1:
another plant, the aluminium corporation of india, came into existence after the war.
['डाला', 'an', '६६७६', 'विजे', 'गढ़वाल', 'हिट', 'टॉप', 'िर', '२००१', 'ताकि', 'आहार', 'िपि', 'वालों', 'पाठक', 'अमृत', 'पारस्परिक', 'माइक्रोस', 'शिश', 'प्रारम्भिक', 'उतार', '्यालय', 'संगठ', 'राज', 'कृषि', '&', 'निर्धारण', 'ऑनलाइन', 'उपभोक्ता', 'उतार', 'उगा', 'चिकित्सक', 'प्रवासी', 'न्यूटन', 'कान', 'मेजर', '))', 'les', 'साइट', 'विश्वविद्यालयों', 'शुर', 'कर्ण', 'गवर्नर', 'हेतु', 'हाँ', 'वाद्य', 'दोनो', 'प्रतिपादन', 'emb', 'जहाँ', 'मार्']


Example 2:
He is doing very good these days
['डाला', 'an', '६६७६', 'इंट', 'फ़िल्म', 'लभ', 'सामूहिक', 'रिलीज़', 'right', 'जुल', 'प्रति', 'परिवार', 'ंकार', 'भौ', 'या', 'जान', 'मार्शल', 'गवर्नमेंट', 'सहायक', 'श्रे', 'समुचित', 'ग्लो', 'सक्षम', 'लाने', 'गाँ', 'तस्वी', 'बनाए', 'ंडित', 'सश', 'पश्चिमी', 'डब्ल्यू', 'पहच', 'ूरत', 'युक्त', 'ऊँचे', 'अभिया', 'घर', 'लेने', 'ames', 'विश्वविद्यालयों', 'शुर', 'बोध', 'arn', 'वालों', 'वाशिंगटन', 'आईटी', 'हाइड्रोजन', 'झ', 'करेगा

In [39]:
# training
for epoch in range(num_epochs):
    model.train()
    losses.append(train(train_loader, model, optimizer, scaler, scheduler, loss_fn, epoch))
    
    # save checkpoint
    checkpoint = {
        'state_dict': model.state_dict(),
        'optimizer':optimizer.state_dict()
    }
    save_checkpoint(checkpoint)
    
    # check accuracy  on test set
    infer(sentences, model, max_seq_len)

  0%|          | 0/352 [00:00<?, ?it/s]/tmp/ipykernel_155/4043967710.py:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(): # for gradient underflowing and overflowing and it makes training faster by converting all floats to float16
100%|██████████| 352/352 [00:56<00:00,  6.24it/s, epoch=0, loss=5.99]


-> Saving CheckPoint
Example 1:
another plant, the aluminium corporation of india, came into existence after the war.
['एक', 'व्यक्ति', 'के', 'कारण', ',', '0000', 'में', ',', 'एक', 'दूसरे', 'की', 'गई', '।', '']


Example 2:
He is doing very good these days
['उसने', 'कहा', ',', '"', '']


Example 3:
this guy is totally mad
['यह', 'है', 'कि', 'यह', 'है', 'कि', '']


Example 4:
what were you saying that day?
['क्या', 'आप', 'इस', 'पर', 'आप', 'क्या', 'है', '?', '']




100%|██████████| 352/352 [00:56<00:00,  6.26it/s, epoch=1, loss=4.71]


-> Saving CheckPoint
Example 1:
another plant, the aluminium corporation of india, came into existence after the war.
['एक', 'भारत', 'के', 'बारे', 'में', ',', 'भारत', 'के', 'बाद', 'भारत', 'में', ',', 'भारत', 'के', 'बारे', 'में', ',', 'भारत', 'का', 'एक', 'और', 'है', '।', '']


Example 2:
He is doing very good these days
['ये', 'सब', 'कुछ', 'भी', 'है', '।', '']


Example 3:
this guy is totally mad
['यह', 'एक', 'है', '']


Example 4:
what were you saying that day?
['क्या', 'तुम', 'ने', 'तुम्', 'ह', 'ें', 'क्या', 'है', '?', '']




100%|██████████| 352/352 [00:56<00:00,  6.26it/s, epoch=2, loss=4.57]


...

-> Saving CheckPoint
Example 1:
another plant, the aluminium corporation of india, came into existence after the war.
['एक', 'बार', 'श्रेणी', 'के', 'पीछे', 'विश', '्राम', 'के', 'पीछे', 'होने', 'के', 'पीछे', 'विश', '्राम', ',', 'भारत', 'के', 'पीछे', 'प्रा', 'धिकारी', 'के', 'पीछे', 'निकले', '-', '']


Example 2:
He is doing very good these days
['वह', 'तो', 'ये', 'बहुत', 'अच्छा', 'है', '।', '']


Example 3:
this guy is totally mad
['यह', 'प्रकृति', 'एक', 'दम', 'निरा', 'है', '']


Example 4:
what were you saying that day?
['तुम्', 'हारी', 'दिन', 'कह', 'ली', 'गयी', 'कि', 'तु', 'झे', 'क्या', 'फ़', 'री', 'ही', 'हुई', 'थीं', '?', '']




100%|██████████| 352/352 [00:56<00:00,  6.29it/s, epoch=18, loss=0.33] 


-> Saving CheckPoint
Example 1:
another plant, the aluminium corporation of india, came into existence after the war.
['एक', 'और', 'निगम', 'भारत', 'के', 'स्वतंत्रता', 'निगम', 'बाद', 'भारत', 'के', 'संरक्षण', 'में', 'बै', 'ठा', 'की', 'है', '।', '']


Example 2:
He is doing very good these days
['वह', 'तो', 'बड़ी', 'सफलता', 'बहुत', 'अच्छा', 'रहता', 'है', '।', '']


Example 3:
this guy is totally mad
['यह', 'हमारी', 'स', 'ोप', 'रि', 'ढ़', 'है', '']


Example 4:
what were you saying that day?
['तुम्', 'ह', 'ें', 'क्या', 'हो', 'गया', 'दिन', '?', '']




100%|██████████| 352/352 [00:56<00:00,  6.29it/s, epoch=19, loss=0.247]


-> Saving CheckPoint
Example 1:
another plant, the aluminium corporation of india, came into existence after the war.
['एक', ',', 'भारत', 'के', 'स्वतंत्रता', ',', 'भारत', 'के', 'पीछे', 'हट', 'कर', 'ने', 'ही', 'महर्षि', 'दिए', '।', '']


Example 2:
He is doing very good these days
['ये', 'च', 'ाली', 'बहुत', 'अच्छी', 'बात', 'रखता', 'है', '']


Example 3:
this guy is totally mad
['यह', 'हमारी', 'स्वा', 'भाव', 'ित', 'है', '']


Example 4:
what were you saying that day?
['तुम्', 'हारी', 'दिन', 'बहुत', 'दिन', '?', '']


